# Creating inference input for fabind
This Jupyter notebook goes through the basic process of preparing an input .csv and files for FABind, using the BDB2020+ benchmark dataset as an example.

In [1]:
import pandas as pd
import os
from tqdm import tqdm
from rdkit import Chem
from io import StringIO
import sys

In [2]:
input_data_dir = 'BDB2020+/dataset/'

In [3]:
# unzip the provided BDB2020+ dataset before proceeding
# note that the provided dataset already provides SMILES strings for the ligands,
# we're simply recreating them here for the sake of consistency / future-proofing

pdb_ids = []
ligand_ids = []
cleaned_smiles = []

for dir_ in tqdm(os.listdir(input_data_dir)):

    protein_pdb = os.path.join(input_data_dir, dir_, 'protein.pdb')
    ligand_pdb = os.path.join(input_data_dir, dir_, 'ligand.pdb')
    ligand_sdf = os.path.join(input_data_dir, dir_, 'ligand.sdf')

    # convert ligand pdb to smiles
    os.system(f'obabel -ipdb {ligand_pdb} -osmi -O {ligand_pdb.replace("pdb", "smi")}')
    
    pdb_ids.append(protein_pdb)
    ligand_ids.append(ligand_sdf)

    with open(ligand_pdb.replace("pdb", "smi"), 'r') as f:
        try:
            cleaned_smiles.append(f.read().strip().split()[0])
        except Exception as e:
            print(e)
            cleaned_smiles.append('')

  0%|          | 0/136 [00:00<?, ?it/s]1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is BDB2020+/dataset/8DD3/ligand.pdb)

1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is BDB2020+/dataset/5SH9/ligand.pdb)

1 molecule converted
  4%|▎         | 5/136 [00:00<00:02, 46.37it/s]1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is BDB2020+/dataset/7DFL/ligand.pdb)

1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is BDB2020+/dataset/7X5H/ligand.pdb)

1 molecule converted
  7%|▋         | 10/136 [00:00<00:02, 44.64it/s]1 molecule converted
1 molecule 

list index out of range


1 molecule converted
1 molecule converted
1 molecule converted
 90%|████████▉ | 122/136 [00:03<00:00, 29.42it/s]1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
 93%|█████████▎| 126/136 [00:03<00:00, 30.28it/s]1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
 96%|█████████▋| 131/136 [00:03<00:00, 33.89it/s]1 molecule converted
1 molecule converted
1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is BDB2020+/dataset/5SGX/ligand.pdb)

1 molecule converted
1 molecule converted
100%|██████████| 136/136 [00:03<00:00, 35.62it/s]


In [4]:
# make dir called pdb_files and copy (or move, for large datasets) all PDB files into it, under name {dir_}.pdb
os.makedirs('pdb_files', exist_ok=True)
for pdb in tqdm(pdb_ids):
    os.system(f'cp {pdb} pdb_files/{pdb.split("/")[-2]}.pdb')

100%|██████████| 136/136 [00:00<00:00, 721.98it/s]


In [5]:
# helper functions for renumbering atoms in the SDF files, from TANKBind
# to match the atom order in the SMILES strings

def read_mol(sdf_fileName, mol2_fileName, verbose=False):
    stderr = sys.stderr
    sio = sys.stderr = StringIO()
    mol = Chem.MolFromMolFile(sdf_fileName, sanitize=False)
    problem = False
    try:
        Chem.SanitizeMol(mol)
        mol = Chem.RemoveHs(mol)
        sm = Chem.MolToSmiles(mol)
    except Exception as e:
        sm = str(e)
        problem = True
    if problem:
        mol = Chem.MolFromMol2File(mol2_fileName, sanitize=False)
        problem = False
        try:
            Chem.SanitizeMol(mol)
            mol = Chem.RemoveHs(mol)
            sm = Chem.MolToSmiles(mol)
            problem = False
        except Exception as e:
            sm = str(e)
            problem = True

    if verbose:
        print(sio.getvalue())
    sys.stderr = stderr
    return mol, problem


def write_renumbered_sdf(toFile, sdf_fileName, mol2_fileName):
    # read in mol
    mol, _ = read_mol(sdf_fileName, mol2_fileName)
    # reorder the mol atom number as in smiles.
    m_order = list(mol.GetPropsAsDict(includePrivate=True, includeComputed=True)['_smilesAtomOutputOrder'])
    mol = Chem.RenumberAtoms(mol, m_order)
    w = Chem.SDWriter(toFile)
    w.write(mol)
    w.close()

In [6]:
# renumber the atoms in the ligands to match smiles order
for ligand in tqdm(ligand_ids):
    write_renumbered_sdf(ligand.replace('.sdf', '_renumbered.sdf'), ligand, ligand.replace('.sdf', '.mol2'))

100%|██████████| 136/136 [00:00<00:00, 1235.74it/s]


In [8]:
# make a dir called gt_mol_files and copy all ligand files into it, under name {dir_}/{dir_}_ligand.sdf

os.makedirs('gt_mol_files', exist_ok=True)
for ligand in tqdm(ligand_ids):
    ligand = ligand.replace('.sdf', '_renumbered.sdf')
    os.makedirs(f'gt_mol_files/{ligand.split("/")[-2]}', exist_ok=True)
    os.system(f'cp {ligand} gt_mol_files/{ligand.split("/")[-2]}/{ligand.split("/")[-2]}_ligand.sdf')

100%|██████████| 136/136 [00:00<00:00, 873.33it/s]


In [9]:
data = pd.DataFrame(columns=['Cleaned_SMILES','pdb_id','ligand_id'])
data['Cleaned_SMILES'] = cleaned_smiles

data['pdb_id'] = os.listdir('gt_mol_files')
data['ligand_id'] = os.listdir('gt_mol_files')

In [10]:
data.to_csv('BDB2020+.csv', index=False)